Sistema mas eficiente

In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv(r'D:\OneDrive\Escritorio\Proyecto 1\Proyecto-Individual-1-ML\data procesada para funciones\data_ml')
df=df[['Id_Pelicula','title', 'coleccion', 'overview', 'genero_1']]
df=df.fillna('')
df=df.iloc[:6000,:]
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Id_Pelicula  6000 non-null   object
 1   title        6000 non-null   object
 2   coleccion    6000 non-null   object
 3   overview     6000 non-null   object
 4   genero_1     6000 non-null   object
dtypes: object(5)
memory usage: 234.5+ KB


In [3]:
df['combined_features'] = df['overview'] + ' ' + df['genero_1'] + ' ' + df['coleccion']
df['combined_features']

0       Led by Woody, Andy's toys live happily in his ...
1       When siblings Judy and Peter discover an encha...
2       A family wedding reignites the ancient feud be...
3       Cheated on, mistreated and stepped on, the wom...
4       Just when George Banks has recovered from his ...
                              ...                        
5995    Pretending to do research for his newspaper, c...
5996    A sobering mid-life crisis fuels dissatisfacti...
5997    An American writer in Rome is stalked by a ser...
5998    Get set for all the zany fun in this hilarious...
5999    From deep within the tomb of the Egyptian phar...
Name: combined_features, Length: 6000, dtype: object

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
matriz_tfidf_1 = tfidf.fit_transform(df['combined_features'])

In [5]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(matriz_tfidf_1, matriz_tfidf_1)

In [6]:
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

def get_recomendacion(title, cosine_sim=cosine_sim):
    if title not in indices:
        return "El titulo ingresado no se encuentra en el dataset, por favor vuelva a intentar"
    idx = indices[title]

    sim_scores = list(enumerate(cosine_sim[idx]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:6]

    movie_indices = [i[0] for i in sim_scores]

    return df['title'].iloc[movie_indices]

In [7]:
print(get_recomendacion('Star Wars'))

1152    The Empire Strikes Back
1165         Return of the Jedi
309           The Swan Princess
1265               Mad Dog Time
5183        The Triumph of Love
Name: title, dtype: object


In [8]:
df.drop(['coleccion','overview','genero_1'],axis=1,inplace=True)
df

,Id_Pelicula,title,combined_features
0,862.0,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,8844.0,Jumanji,When siblings Judy and Peter discover an encha...
2,15602.0,Grumpier Old Men,A family wedding reignites the ancient feud be...
3,31357.0,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom..."
4,11862.0,Father of the Bride Part II,Just when George Banks has recovered from his ...
...,...,...,...
5995,48313.0,Tag: The Assassination Game,"Pretending to do research for his newspaper, c..."
5996,48249.0,Tempest,A sobering mid-life crisis fuels dissatisfacti...
5997,29702.0,Tenebre,An American writer in Rome is stalked by a ser...
5998,24830.0,They Call Me Bruce?,Get set for all the zany fun in this hilarious...


In [9]:
#np.save('cosine_sim.npy', cosine_sim)
#df.to_csv('data_funcion_recomendacion')